# Day 4: Building Custom MCP Servers with FastMCP

## Learning Objectives
By the end of this lesson, you will be able to:
- Understand the fundamentals of MCP server development
- Build custom MCP servers using the FastMCP Python SDK
- Implement proper tool registration patterns
- Apply authentication and authorization strategies
- Optimize MCP server performance
- Connect LangGraph agents to custom MCP servers

---

## 4.1 Introduction to Model Context Protocol (MCP)

The **Model Context Protocol (MCP)** is an open protocol that enables seamless integration between LLM applications and external data sources and tools. Introduced by Anthropic in November 2024, MCP provides a standardized way to connect AI systems to resources and functionality, often described as "the USB-C port for AI."

### What is MCP and Why Does It Matter?

Before MCP, every AI application had to build custom integrations for external tools and data sources. This led to:
- **Fragmented ecosystem** - Each app implemented its own tool interfaces
- **Vendor lock-in** - Tools were tied to specific AI platforms
- **Development overhead** - Teams spent more time on integrations than core features
- **Limited interoperability** - Difficult to share tools across different AI systems

MCP solves these problems by providing a universal protocol that enables:

### Key MCP Components

MCP defines **three main primitives** that form the foundation of all AI-tool interactions:

#### 1. **Tools** 🔧
Executable functions that AI agents can invoke to perform actions or fetch information.
- **Purpose**: Enable AI to take actions in the world
- **Examples**: Send emails, query databases, perform calculations, make API calls
- **Characteristics**: Have defined parameters, return structured results, can modify state

#### 2. **Resources** 📚  
Structured data that can be read by AI systems, similar to GET endpoints.
- **Purpose**: Provide context and information to AI
- **Examples**: Documentation, configuration files, datasets, knowledge bases
- **Characteristics**: Read-only, versioned, can be cached, provide metadata

#### 3. **Prompts** 💬
Reusable templates for LLM interactions that can be invoked with parameters.
- **Purpose**: Standardize common interaction patterns
- **Examples**: Code review templates, writing assistants, analysis frameworks
- **Characteristics**: Parameterized, reusable, consistent formatting

### MCP Architecture: Client-Server Model

```
┌─────────────────┐    MCP Protocol    ┌─────────────────┐
│   LLM Client    │◄──────────────────►│   MCP Server    │
│  (LangGraph)    │                    │   (FastMCP)     │
│                 │                    │                 │
│ • Agent Logic   │                    │ • Tools         │
│ • Conversations │                    │ • Resources     │
│ • Decision      │                    │ • Business      │
│   Making        │                    │   Logic         │
└─────────────────┘                    └─────────────────┘
```

**Key Benefits:**
- **Standardization**: Universal interface for all AI-tool connections
- **Security**: Built-in authentication, authorization, and sandboxing
- **Scalability**: Efficient client-server architecture supports many concurrent requests
- **Flexibility**: Multiple transport protocols (stdio, HTTP, WebSocket)
- **Composability**: Mix and match tools from different vendors seamlessly

### Real-World MCP Applications

**Enterprise Integration:**
- Connect agents to CRM systems, databases, and internal APIs
- Automate workflows across multiple business applications
- Provide AI access to company knowledge bases and documentation

**Development Tools:**
- AI coding assistants with access to version control, testing, and deployment
- Automated code review and documentation generation
- Integration with IDEs, build systems, and monitoring tools

**Data Science & Analytics:**
- AI agents that can query data warehouses and visualization tools
- Automated report generation and insight discovery
- Integration with ML pipelines and experiment tracking

### MCP vs. Other Approaches

| Approach | Pros | Cons | Best For |
|----------|------|------|----------|
| **Direct API Integration** | Simple, direct control | Fragmented, hard to maintain | Single-tool apps |
| **Function Calling** | Built into LLMs | Limited, vendor-specific | Simple tool usage |
| **MCP** | Standardized, scalable, secure | Learning curve | Production systems |

### Why Build MCP Servers?

Building MCP servers enables you to:

1. **Future-proof your tools** - Work with any MCP-compatible AI client
2. **Reduce integration overhead** - One server, many client applications  
3. **Enable agent ecosystems** - Compose complex workflows from simple tools
4. **Maintain security boundaries** - Controlled access to sensitive systems
5. **Scale efficiently** - Handle multiple AI agents with one server instance

In this lesson, you'll learn to build production-ready MCP servers using **FastMCP** and integrate them with **LangGraph agents** for powerful AI workflows.

## 4.2 Setting Up Your MCP Development Environment

Building MCP servers requires a carefully configured development environment. We'll use **FastMCP 2.0** - the actively maintained Python SDK that provides a complete toolkit for the MCP ecosystem.

### Installation Strategy

We'll install packages in this order to avoid dependency conflicts:

1. **Core MCP Development**: FastMCP for server creation
2. **LangGraph Integration**: Official adapters for production-ready agent integration  
3. **LLM Providers**: Support for OpenAI and Anthropic models
4. **Supporting Libraries**: HTTP clients, async utilities, and development tools

### Development Environment Setup

The setup includes two main components:
- **MCP Server Development**: Using FastMCP to create tools and resources
- **Agent Integration**: Using LangGraph with MCP adapters for real agent workflows

**Note**: This lesson uses working examples that you can run immediately. All examples work with either OpenAI or Anthropic API keys.

In [1]:
# Complete MCP Development Environment Setup

# Install required packages
%%capture --no-stderr
%pip install fastmcp mcp httpx langgraph langchain-mcp-adapters langchain-openai

In [2]:
# Core MCP development imports
from fastmcp import FastMCP
import asyncio
import httpx
import json
import time
import os
from typing import List, Dict, Optional, Any

# LangGraph and MCP integration imports
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [3]:
# prompt: prompt user for openai api key and set llm variable to use the gpt-4.1-mini model
from langchain_openai import ChatOpenAI
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass('OPENAI_API_KEY:\t')

llm = ChatOpenAI(model="gpt-4.1-mini")

print(f"Using LLM model: {llm.model_name}")

OPENAI_API_KEY:	··········
Using LLM model: gpt-4.1-mini


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


### Your First MCP Server: Step-by-Step

Let's build a simple but complete MCP server to understand the core concepts. This server will provide mathematical operations that AI agents can use.

**Key FastMCP Concepts:**
- `FastMCP("ServerName")` - Creates a new MCP server
- `@mcp.tool()` - Decorator that registers a function as an MCP tool
- **Automatic Schema Generation** - FastMCP infers JSON schemas from Python type hints
- **Error Handling** - Tools should return structured error information, not raise exceptions

**Design Principles:**
1. **Single Responsibility** - Each tool does one thing well
2. **Clear Documentation** - Docstrings help AI understand when to use tools
3. **Robust Validation** - Always validate inputs and handle edge cases
4. **Structured Responses** - Return consistent, well-formatted data

In [4]:
# Create your first MCP server - Enhanced Math Operations
math_mcp = FastMCP("EnhancedMathServer")

@math_mcp.tool(title="Addition Function")
def add_numbers(a: float, b: float) -> dict:
    """Add two numbers and return detailed result information.

    This tool provides addition with comprehensive metadata that helps
    AI agents understand the operation and verify results.

    Args:
        a: First number (any real number)
        b: Second number (any real number)

    Returns:
        Dictionary with operation details, result, and metadata
    """
    result = a + b
    return {
        "operation": "addition",
        "expression": f"{a} + {b}",
        "result": result,
        "operands": {"a": a, "b": b},
        "operation_type": "arithmetic",
        "properties": {
            "commutative": True,  # a + b = b + a
            "associative": True,  # (a + b) + c = a + (b + c)
            "identity_element": 0  # a + 0 = a
        }
    }

@math_mcp.tool()
def multiply_numbers(a: float, b: float) -> dict:
    """Multiply two numbers with comprehensive result analysis.

    Args:
        a: First number (multiplicand)
        b: Second number (multiplier)

    Returns:
        Dictionary with multiplication details and mathematical properties
    """
    result = a * b

    # Analyze the result
    analysis = {
        "is_positive": result > 0,
        "is_negative": result < 0,
        "is_zero": result == 0,
        "magnitude": abs(result)
    }

    # Special cases
    special_cases = []
    if a == 0 or b == 0:
        special_cases.append("multiplication_by_zero")
    if a == 1:
        special_cases.append("multiplication_by_identity")
    if b == 1:
        special_cases.append("multiplication_by_identity")
    if a == -1:
        special_cases.append("multiplication_by_negative_one")
    if b == -1:
        special_cases.append("multiplication_by_negative_one")

    return {
        "operation": "multiplication",
        "expression": f"{a} × {b}",
        "result": result,
        "operands": {"multiplicand": a, "multiplier": b},
        "analysis": analysis,
        "special_cases": special_cases,
        "properties": {
            "commutative": True,  # a × b = b × a
            "associative": True,  # (a × b) × c = a × (b × c)
            "identity_element": 1  # a × 1 = a
        }
    }

@math_mcp.tool()
def calculate_percentage(value: float, percentage: float) -> dict:
    """Calculate what percentage of a value represents, with validation.

    Args:
        value: The base value (any real number)
        percentage: The percentage (0-100 for normal percentages)

    Returns:
        Dictionary with percentage calculation and interpretations
    """
    # Validate percentage range (allow negative for calculations)
    if percentage < -1000 or percentage > 1000:
        return {
            "error": "Percentage out of reasonable range (-1000% to 1000%)",
            "provided_percentage": percentage,
            "valid_range": "Typically 0-100, extended range -1000 to 1000 allowed"
        }

    percentage_value = (value * percentage) / 100

    # Provide multiple interpretations
    interpretations = {
        "percentage_of_value": f"{percentage}% of {value} is {percentage_value}",
        "fraction_form": f"{percentage}/100 × {value} = {percentage_value}",
        "decimal_form": f"{percentage/100} × {value} = {percentage_value}"
    }

    # Context about the calculation
    context = {
        "is_increase": percentage > 0 and value > 0,
        "is_decrease": percentage < 0 and value > 0,
        "percentage_greater_than_100": percentage > 100,
        "calculated_value_larger_than_original": abs(percentage_value) > abs(value)
    }

    return {
        "operation": "percentage_calculation",
        "expression": f"{percentage}% of {value}",
        "result": percentage_value,
        "inputs": {"value": value, "percentage": percentage},
        "interpretations": interpretations,
        "context": context
    }

@math_mcp.resource("resource://server-capabilities")
def get_server_capabilities() -> dict:
    """Get information about this MCP server's mathematical capabilities.

    Returns:
        Dictionary describing available operations and server metadata
    """
    return {
        "server_name": "EnhancedMathServer",
        "version": "1.0.0",
        "capabilities": {
            "basic_arithmetic": ["addition", "multiplication"],
            "percentage_calculations": ["percentage_of_value"],
            "advanced_features": [
                "detailed_operation_metadata",
                "mathematical_property_analysis",
                "input_validation_and_error_handling",
                "special_case_detection"
            ]
        },
        "supported_number_types": ["integers", "floating_point", "negative_numbers"],
        "output_format": "structured_json_with_metadata",
        "error_handling": "graceful_with_detailed_messages",
        "mathematical_properties": {
            "addition": {"commutative": True, "associative": True, "identity": 0},
            "multiplication": {"commutative": True, "associative": True, "identity": 1}
        }
    }

print("🔧 Enhanced Math MCP Server created with advanced capabilities:")
print("📊 Available tools:")
print("   • add_numbers: Addition with mathematical properties")
print("   • multiply_numbers: Multiplication with result analysis")
print("   • calculate_percentage: Percentage calculations with validation")
print("   • get_server_capabilities: Server introspection and documentation")
print("\n✨ Features:")
print("   • Comprehensive input validation")
print("   • Detailed mathematical analysis")
print("   • Special case detection")
print("   • Educational mathematical properties")
print("   • Structured error handling")

🔧 Enhanced Math MCP Server created with advanced capabilities:
📊 Available tools:
   • add_numbers: Addition with mathematical properties
   • multiply_numbers: Multiplication with result analysis
   • calculate_percentage: Percentage calculations with validation
   • get_server_capabilities: Server introspection and documentation

✨ Features:
   • Comprehensive input validation
   • Detailed mathematical analysis
   • Special case detection
   • Educational mathematical properties
   • Structured error handling


In [5]:
from fastmcp import Client

client = Client(math_mcp)

async with client:
  tools = await client.list_tools()
  resources = await client.list_resources()
  for tool in tools:
    print(tool.name, tool.description, tool.title)
  print("-" * 20)
  for resource in resources:
    print(resource.name, resource.description)

add_numbers Add two numbers and return detailed result information.

This tool provides addition with comprehensive metadata that helps
AI agents understand the operation and verify results.

Args:
    a: First number (any real number)
    b: Second number (any real number)

Returns:
    Dictionary with operation details, result, and metadata Addition Function
multiply_numbers Multiply two numbers with comprehensive result analysis.

Args:
    a: First number (multiplicand)
    b: Second number (multiplier)

Returns:
    Dictionary with multiplication details and mathematical properties None
calculate_percentage Calculate what percentage of a value represents, with validation.

Args:
    value: The base value (any real number)
    percentage: The percentage (0-100 for normal percentages)

Returns:
    Dictionary with percentage calculation and interpretations None
--------------------
get_server_capabilities Get information about this MCP server's mathematical capabilities.

Returns:
 

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## 4.3 Tool Registration Patterns

When creating tools, follow these best practices for clarity and safety:

### 1. Clear Names and Descriptions
- Give each tool a clear name and description
- Use docstrings - the AI will see these when deciding to use a tool

### 2. Define Input Parameters with Schemas
- FastMCP infers JSON schema from type hints
- You can specify complex schemas for structured inputs

### 3. Input Validation
- Always validate inputs inside your tool functions
- Don't trust that the AI will always send perfect data

### 4. Keep Tools Focused (UNIX Philosophy)
- One tool = one task
- Makes it easier for AI to choose the right tool

In [6]:
# Advanced MCP server with better tool patterns
advanced_mcp = FastMCP("AdvancedMathServer")

@advanced_mcp.tool()
def solve_quadratic(a: float, b: float, c: float) -> dict:
    """Solve a quadratic equation ax² + bx + c = 0.

    Args:
        a: Coefficient of x² (must not be zero)
        b: Coefficient of x
        c: Constant term

    Returns:
        Dictionary with solutions and discriminant info
    """
    # Input validation
    if a == 0:
        raise ValueError("Coefficient 'a' cannot be zero for a quadratic equation")

    # Calculate discriminant
    discriminant = b**2 - 4*a*c

    result = {
        "equation": f"{a}x² + {b}x + {c} = 0",
        "discriminant": discriminant
    }

    if discriminant > 0:
        import math
        sqrt_discriminant = math.sqrt(discriminant)
        x1 = (-b + sqrt_discriminant) / (2*a)
        x2 = (-b - sqrt_discriminant) / (2*a)
        result.update({
            "solutions": [x1, x2],
            "type": "two_real_solutions"
        })
    elif discriminant == 0:
        x = -b / (2*a)
        result.update({
            "solutions": [x],
            "type": "one_real_solution"
        })
    else:
        result.update({
            "solutions": [],
            "type": "no_real_solutions"
        })

    return result

@advanced_mcp.tool()
def calculate_statistics(numbers: List[float]) -> dict:
    """Calculate basic statistics for a list of numbers.

    Args:
        numbers: List of numbers to analyze (must not be empty)

    Returns:
        Dictionary with mean, median, mode, std_dev, and range
    """
    # Input validation
    if not numbers:
        raise ValueError("Numbers list cannot be empty")

    if not all(isinstance(n, (int, float)) for n in numbers):
        raise ValueError("All elements must be numbers")

    import statistics

    sorted_numbers = sorted(numbers)

    result = {
        "count": len(numbers),
        "mean": statistics.mean(numbers),
        "median": statistics.median(numbers),
        "min": min(numbers),
        "max": max(numbers),
        "range": max(numbers) - min(numbers)
    }

    # Calculate standard deviation if we have more than one number
    if len(numbers) > 1:
        result["std_dev"] = statistics.stdev(numbers)
    else:
        result["std_dev"] = 0

    # Try to calculate mode (may not exist for all datasets)
    try:
        result["mode"] = statistics.mode(numbers)
    except statistics.StatisticsError:
        result["mode"] = None

    return result

print("🔧 Advanced Math MCP Server created with tools:")
print("- solve_quadratic")
print("- calculate_statistics")

🔧 Advanced Math MCP Server created with tools:
- solve_quadratic
- calculate_statistics


In [7]:
from fastmcp import Client

client = Client(advanced_mcp)

async with client:
  tools = await client.list_tools()
  resources = await client.list_resources()
  for tool in tools:
    print(tool.name, tool.description, tool.title)
  print("-" * 20)
  for resource in resources:
    print(resource.name, resource.description)

solve_quadratic Solve a quadratic equation ax² + bx + c = 0.

Args:
    a: Coefficient of x² (must not be zero)
    b: Coefficient of x
    c: Constant term

Returns:
    Dictionary with solutions and discriminant info None
calculate_statistics Calculate basic statistics for a list of numbers.

Args:
    numbers: List of numbers to analyze (must not be empty)

Returns:
    Dictionary with mean, median, mode, std_dev, and range None
--------------------


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## 4.4 Resources: Read-Only Data Access

In addition to tools, MCP supports **resources** - read-only data that can be accessed by AI systems. Resources are like GET endpoints that provide structured data.

In [8]:
# Example MCP server with resources
resource_mcp = FastMCP("ResourceServer")

# Sample data for resources
MATH_FORMULAS = {
    "quadratic": {
        "formula": "x = (-b ± √(b² - 4ac)) / 2a",
        "description": "Quadratic formula for solving ax² + bx + c = 0",
        "example": "For x² - 5x + 6 = 0: a=1, b=-5, c=6"
    },
    "distance": {
        "formula": "d = √((x₂-x₁)² + (y₂-y₁)²)",
        "description": "Distance between two points in 2D space",
        "example": "Distance between (0,0) and (3,4) = 5"
    },
    "compound_interest": {
        "formula": "A = P(1 + r/n)^(nt)",
        "description": "Compound interest calculation",
        "example": "P=principal, r=rate, n=compounds per year, t=time"
    }
}

@resource_mcp.resource("math-formula://{formula_name}")
def get_math_formula(formula_name: str) -> str:
    """Get mathematical formula information by name.

    Available formulas: quadratic, distance, compound_interest
    """
    if formula_name not in MATH_FORMULAS:
        return f"Formula '{formula_name}' not found. Available: {list(MATH_FORMULAS.keys())}"

    formula_info = MATH_FORMULAS[formula_name]
    return f"""
Formula: {formula_info['formula']}
Description: {formula_info['description']}
Example: {formula_info['example']}
""".strip()

@resource_mcp.resource("math-constants://")
def get_math_constants() -> str:
    """Get common mathematical constants."""
    import math
    constants = {
        "π (pi)": math.pi,
        "e (Euler's number)": math.e,
        "φ (Golden ratio)": (1 + math.sqrt(5)) / 2,
        "√2": math.sqrt(2),
        "√3": math.sqrt(3)
    }

    result = "Mathematical Constants:\n"
    for name, value in constants.items():
        result += f"  {name}: {value:.10f}\n"

    return result

print("📚 Resource MCP Server created with resources:")
print("- math-formula://{formula_name}")
print("- math-constants://")

📚 Resource MCP Server created with resources:
- math-formula://{formula_name}
- math-constants://


In [9]:
client = Client(resource_mcp)

async with client:
  tools = await client.list_tools()
  resources = await client.list_resources()
  templated_resources = await client.list_resource_templates()
  for tool in tools:
    print(tool.name, tool.description, tool.title)
  print("-" * 40)
  for resource in resources:
    print(resource.name, resource.description)
  print("-" * 40)
  for template in templated_resources:
    print(template.name, template.description)

----------------------------------------
get_math_constants Get common mathematical constants.
----------------------------------------
get_math_formula Get mathematical formula information by name.

Available formulas: quadratic, distance, compound_interest


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## 4.5 Authentication & Authorization Strategies

By default, MCP does not enforce authentication - it's up to us to secure our servers. **Never deploy an MCP server without access control in production.**

### Common Security Strategies:

1. **API Keys or Tokens**: Simple authentication mechanism
2. **OAuth2**: For user context and enterprise SSO
3. **Role-Based Access Control (RBAC)**: Different permissions for different users
4. **Tool-level Authorization**: Per-tool permission checks

In [10]:
# Example: MCP Server with API Key Authentication
import os
from functools import wraps

# Secure MCP server with authentication
secure_mcp = FastMCP("SecureMathServer")

# Environment variable for API key (in production, use proper secret management)
VALID_API_KEY = os.getenv("MCP_API_KEY", "demo-key-12345")

def require_api_key(func):
    """Decorator to require API key authentication."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        # In a real MCP server, you'd get this from the request context
        # For demo purposes, we'll simulate it
        provided_key = kwargs.pop('api_key', None)

        if not provided_key or provided_key != VALID_API_KEY:
            raise PermissionError("Invalid or missing API key")

        return func(*args, **kwargs)
    return wrapper

# User roles for RBAC
USER_ROLES = {
    "admin": ["read", "write", "compute"],
    "user": ["read", "compute"],
    "guest": ["read"]
}

def require_permission(permission):
    """Decorator to require specific permission."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            user_role = kwargs.pop('user_role', 'guest')

            if permission not in USER_ROLES.get(user_role, []):
                raise PermissionError(f"Role '{user_role}' does not have '{permission}' permission")

            return func(*args, **kwargs)
        return wrapper
    return decorator

# Secure tools with authentication and authorization
@secure_mcp.tool()
@require_permission("read")
def get_formula_info(formula_name: str, user_role: str = "guest") -> str:
    """Get formula information (requires 'read' permission)."""
    formulas = {
        "area_circle": "A = πr²",
        "pythagorean": "a² + b² = c²",
        "slope": "m = (y₂ - y₁) / (x₂ - x₁)"
    }
    return formulas.get(formula_name, "Formula not found")

@secure_mcp.tool()
@require_permission("compute")
def secure_calculate(expression: str, user_role: str = "guest") -> dict:
    """Safely evaluate mathematical expressions (requires 'compute' permission)."""
    import ast
    import operator

    # Safe evaluation - only allow basic math operations
    allowed_operators = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.Pow: operator.pow,
        ast.USub: operator.neg,
        ast.UAdd: operator.pos,
    }

    def safe_eval(node):
        if isinstance(node, ast.Constant):  # numbers
            return node.value
        elif isinstance(node, ast.BinOp):  # binary operations
            return allowed_operators[type(node.op)](safe_eval(node.left), safe_eval(node.right))
        elif isinstance(node, ast.UnaryOp):  # unary operations
            return allowed_operators[type(node.op)](safe_eval(node.operand))
        else:
            raise ValueError(f"Unsafe operation: {type(node).__name__}")

    try:
        tree = ast.parse(expression, mode='eval')
        result = safe_eval(tree.body)
        return {
            "expression": expression,
            "result": result,
            "user_role": user_role,
            "status": "success"
        }
    except Exception as e:
        return {
            "expression": expression,
            "error": str(e),
            "user_role": user_role,
            "status": "error"
        }

print("🔒 Secure MCP Server created with role-based access control")
print("Available roles: admin, user, guest")
print("Permissions: read, write, compute")

🔒 Secure MCP Server created with role-based access control
Available roles: admin, user, guest
Permissions: read, write, compute


## 4.6 Performance Optimization Techniques

MCP servers should be efficient, especially for real-time agent interactions.

### Key Optimization Strategies:

1. **Async I/O and Batching**: Handle multiple requests concurrently
2. **Caching Results**: Cache expensive operations
3. **Tool Granularity**: Balance between too many small tools vs. too few large ones
4. **Keep Servers Stateless**: Enable horizontal scaling

In [11]:
# Performance-optimized MCP server with caching
import time
import asyncio
from functools import lru_cache
from typing import Dict, Any

optimized_mcp = FastMCP("OptimizedServer")

# Simple in-memory cache with TTL
class TTLCache:
    def __init__(self, ttl_seconds: int = 300):  # 5 minutes default
        self.cache: Dict[str, Dict[str, Any]] = {}
        self.ttl = ttl_seconds

    def get(self, key: str) -> Any:
        if key in self.cache:
            entry = self.cache[key]
            if time.time() - entry['timestamp'] < self.ttl:
                return entry['value']
            else:
                del self.cache[key]
        return None

    def set(self, key: str, value: Any) -> None:
        self.cache[key] = {
            'value': value,
            'timestamp': time.time()
        }

    def clear(self) -> None:
        self.cache.clear()

# Create cache instance
cache = TTLCache(ttl_seconds=60)  # 1 minute cache

@optimized_mcp.tool()
async def expensive_calculation(n: int) -> dict:
    """Simulate an expensive calculation with caching.

    Calculates the sum of squares from 1 to n.
    """
    cache_key = f"sum_squares_{n}"

    # Check cache first
    cached_result = cache.get(cache_key)
    if cached_result is not None:
        return {
            "n": n,
            "result": cached_result,
            "from_cache": True,
            "calculation_time": 0
        }

    # Perform expensive calculation
    start_time = time.time()

    # Simulate expensive work
    await asyncio.sleep(0.1)  # Simulate network delay or complex computation

    result = sum(i**2 for i in range(1, n + 1))

    calculation_time = time.time() - start_time

    # Cache the result
    cache.set(cache_key, result)

    return {
        "n": n,
        "result": result,
        "from_cache": False,
        "calculation_time": calculation_time
    }

@optimized_mcp.tool()
async def batch_fibonacci(numbers: List[int]) -> dict:
    """Calculate Fibonacci numbers for multiple inputs efficiently.

    Uses memoization to avoid recalculating the same values.
    """

    @lru_cache(maxsize=1000)
    def fib(n: int) -> int:
        if n <= 1:
            return n
        return fib(n - 1) + fib(n - 2)

    start_time = time.time()

    results = {}
    for num in numbers:
        if num < 0:
            results[num] = "Error: Fibonacci not defined for negative numbers"
        elif num > 100:  # Prevent extremely large calculations
            results[num] = "Error: Number too large (max 100)"
        else:
            results[num] = fib(num)

    calculation_time = time.time() - start_time

    return {
        "input_numbers": numbers,
        "results": results,
        "calculation_time": calculation_time,
        "cache_info": {
            "hits": fib.cache_info().hits,
            "misses": fib.cache_info().misses
        }
    }

@optimized_mcp.tool()
def get_cache_stats() -> dict:
    """Get cache statistics for monitoring performance."""
    return {
        "cache_size": len(cache.cache),
        "cache_keys": list(cache.cache.keys()),
        "ttl_seconds": cache.ttl
    }

@optimized_mcp.tool()
def clear_cache() -> dict:
    """Clear the cache (admin operation)."""
    cache.clear()
    return {"status": "Cache cleared successfully"}

print("⚡ Optimized MCP Server created with performance features:")
print("- TTL caching for expensive operations")
print("- Async operations")
print("- Memoization for recursive calculations")
print("- Batch processing capabilities")

⚡ Optimized MCP Server created with performance features:
- TTL caching for expensive operations
- Async operations
- Memoization for recursive calculations
- Batch processing capabilities


## 4.7 Production LangGraph Agent Integration

Now we'll create **real LangGraph agents** that connect to our MCP servers using the official `langchain-mcp-adapters` library. This demonstrates the complete production workflow from server development to agent deployment.

### Real-World Integration Architecture

```
┌─────────────────────┐    HTTP/stdio     ┌─────────────────────┐
│   LangGraph Agent   │◄─────────────────►│   Math MCP Server   │
│                     │                   │                     │
│ • GPT-4o-mini or    │    MCP Protocol   │ • add_numbers       │
│   Claude-3.5-Sonnet │◄─────────────────►│ • multiply_numbers  │
│ • MultiServerMCP    │                   │ • calculate_percent │
│   Client            │    HTTP           │ • server_info       │
│ • ReAct Agent       │◄─────────────────►│                     │
│   Pattern           │                   │ Weather MCP Server  │
│                     │                   │ • get_weather       │
│                     │                   │ • forecast          │
│                     │                   │ • compare_cities    │
└─────────────────────┘                   └─────────────────────┘
```

### Key Integration Components

1. **MultiServerMCPClient**: Official adapter for connecting to multiple MCP servers
2. **Transport Protocols**: Support for stdio (local) and HTTP (production) transports
3. **Agent Creation**: Using LangGraph's `create_react_agent` for natural language interaction
4. **Tool Discovery**: Automatic detection and integration of available tools
5. **Multi-Modal Support**: Combine different tool types (math, weather, database, etc.)

### Benefits of This Architecture

- **Scalability**: Each MCP server can be developed and deployed independently
- **Modularity**: Mix and match different tool providers as needed
- **Flexibility**: Easy to add new capabilities without changing agent code
- **Production Ready**: HTTP transport supports load balancing and monitoring
- **Language Agnostic**: MCP servers can be written in any language

### Create Standalone MCP Server for LangGraph Integration

In [12]:
"""
Production Math MCP Server for LangGraph Integration
Standalone server that supports both stdio and HTTP transports
"""
from fastmcp import FastMCP
import math
from typing import Dict, Any

# Create the MCP server
math_server = FastMCP("ProductionMathServer")

@math_server.tool()
def add_numbers(a: float, b: float) -> Dict[str, Any]:
    """Add two numbers with detailed result information."""
    result = a + b
    return {
        "operation": "addition",
        "expression": f"{a} + {b}",
        "result": result,
        "operands": {"a": a, "b": b},
        "properties": {
            "commutative": True,
            "associative": True,
            "identity_element": 0
        }
    }

@math_server.tool()
def multiply_numbers(a: float, b: float) -> Dict[str, Any]:
    """Multiply two numbers with comprehensive analysis."""
    result = a * b

    analysis = {
        "is_positive": result > 0,
        "is_negative": result < 0,
        "is_zero": result == 0,
        "magnitude": abs(result)
    }

    special_cases = []
    if a == 0 or b == 0:
        special_cases.append("multiplication_by_zero")
    if abs(a) == 1:
        special_cases.append("multiplication_by_identity_or_negative_one")
    if abs(b) == 1:
        special_cases.append("multiplication_by_identity_or_negative_one")

    return {
        "operation": "multiplication",
        "expression": f"{a} × {b}",
        "result": result,
        "operands": {"multiplicand": a, "multiplier": b},
        "analysis": analysis,
        "special_cases": special_cases
    }

@math_server.tool()
def calculate_power(base: float, exponent: float) -> Dict[str, Any]:
    """Calculate base raised to the power of exponent with safety checks."""

    # Safety checks for extreme values
    if abs(base) > 1000 and abs(exponent) > 10:
        return {
            "error": "Calculation too large to prevent overflow",
            "base": base,
            "exponent": exponent,
            "max_safe_base": 1000,
            "max_safe_exponent": 10
        }

    try:
        result = base ** exponent

        # Check for overflow
        if abs(result) > 1e10:
            return {
                "error": "Result too large",
                "base": base,
                "exponent": exponent,
                "max_result_magnitude": 1e10
            }

        return {
            "operation": "exponentiation",
            "expression": f"{base}^{exponent}",
            "result": result,
            "base": base,
            "exponent": exponent,
            "special_cases": {
                "base_is_zero": base == 0,
                "exponent_is_zero": exponent == 0,
                "base_is_one": base == 1,
                "exponent_is_one": exponent == 1
            }
        }

    except (OverflowError, ZeroDivisionError) as e:
        return {
            "error": f"Mathematical error: {str(e)}",
            "base": base,
            "exponent": exponent
        }

@math_server.tool()
def solve_quadratic(a: float, b: float, c: float) -> Dict[str, Any]:
    """Solve quadratic equation ax² + bx + c = 0 with comprehensive analysis."""

    if a == 0:
        if b == 0:
            if c == 0:
                return {
                    "equation": "0 = 0",
                    "type": "identity",
                    "solutions": "infinite_solutions"
                }
            else:
                return {
                    "equation": f"{c} = 0",
                    "type": "contradiction",
                    "solutions": "no_solutions"
                }
        else:
            # Linear equation
            solution = -c / b
            return {
                "equation": f"{b}x + {c} = 0",
                "type": "linear",
                "solutions": [round(solution, 6)]
            }

    # Quadratic equation
    discriminant = b**2 - 4*a*c

    result = {
        "equation": f"{a}x² + {b}x + {c} = 0",
        "coefficients": {"a": a, "b": b, "c": c},
        "discriminant": discriminant,
        "vertex": {
            "x": round(-b / (2*a), 6),
            "y": round(c - (b**2) / (4*a), 6)
        }
    }

    if discriminant > 0:
        sqrt_discriminant = math.sqrt(discriminant)
        x1 = (-b + sqrt_discriminant) / (2*a)
        x2 = (-b - sqrt_discriminant) / (2*a)
        result.update({
            "type": "two_real_solutions",
            "solutions": [round(x1, 6), round(x2, 6)]
        })
    elif discriminant == 0:
        x = -b / (2*a)
        result.update({
            "type": "one_real_solution",
            "solutions": [round(x, 6)]
        })
    else:
        real_part = -b / (2*a)
        imaginary_part = math.sqrt(-discriminant) / (2*a)
        result.update({
            "type": "complex_solutions",
            "real_part": round(real_part, 6),
            "imaginary_part": round(imaginary_part, 6),
            "solutions": [
                f"{real_part:.3f} + {imaginary_part:.3f}i",
                f"{real_part:.3f} - {imaginary_part:.3f}i"
            ]
        })

    return result

@math_server.tool()
def calculate_fibonacci(n: int) -> Dict[str, Any]:
    """Calculate the nth Fibonacci number with sequence information."""

    if n < 0:
        return {"error": "Fibonacci sequence not defined for negative numbers"}

    if n > 100:
        return {"error": "Position too large (maximum: 100)"}

    def fib(num):
        if num <= 1:
            return num
        return fib(num - 1) + fib(num - 2)

    result = fib(n)

    # Generate sequence up to position n (limited for performance)
    sequence_length = min(n + 1, 15)
    sequence = [fib(i) for i in range(sequence_length)]

    return {
        "position": n,
        "fibonacci_number": result,
        "sequence": sequence,
        "sequence_note": f"Showing first {sequence_length} numbers" if n >= 15 else "Complete sequence"
    }

### Complete LangGraph Agent Integration with MCP

We'll create a ReAct agent that has access to these tools using the prebuilt LangGraph option and converting the toolkit to Langchain's ecosystem using the `langchain_mcp_adapter`:

In [13]:
from fastmcp import Client
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools

math_client = Client(math_server)

async def math_chat():
  async with math_client:
    tools = await load_mcp_tools(math_client.session)
    agent = create_react_agent(model=llm, tools=tools)
    print("Welcome to Advanced Math Chat!")
    print("To exit, please type 'quit' or 'q'")
    while True:
      user_input = input("User:\t")
      if user_input.lower() in ["quit", "q"]:
        print("Exiting chat.")
        break
      message_template = {"messages": [HumanMessage(content=user_input)]}
      async for message in agent.astream(message_template):
        print(message)

Now let's test our client:

In [ ]:
await math_chat()

Welcome to Advanced Math Chat!
To exit, please type 'quit' or 'q'


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


User:	2+2
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Av2qlvo4BHZIUXlU87QdoVdD', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'add_numbers'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 171, 'total_tokens': 189, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a150906e27', 'id': 'chatcmpl-CEhwhOrCHUEEMDr8YyFv932dbLPyZ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7ee72d13-ee7b-4c97-bcb8-e9aafdea5c3f-0', tool_calls=[{'name': 'add_numbers', 'args': {'a': 2, 'b': 2}, 'id': 'call_Av2qlvo4BHZIUXlU87QdoVdD', 'type': 'tool_call'}], usage_metadata={'input_tokens': 171, 'output_tokens': 18, 'total_tokens': 1

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


{'agent': {'messages': [AIMessage(content='2 + 2 = 4', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 253, 'total_tokens': 261, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a150906e27', 'id': 'chatcmpl-CEhwleOiUcD6v5GU694Z5NQg7GDT3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--4d07a5e7-f491-4cf3-884d-97d19b8397eb-0', usage_metadata={'input_tokens': 253, 'output_tokens': 8, 'total_tokens': 261, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}


## 4.8 Multi-Server Agent Configuration

One of the most powerful features of MCP is the ability to connect agents to multiple servers simultaneously. This enables agents to access diverse capabilities from different sources.

### Multi-Server Architecture Benefits

1. **Modularity**: Each server focuses on specific domain expertise
2. **Scalability**: Servers can be developed and deployed independently  
3. **Flexibility**: Mix and match different tool providers
4. **Fault Tolerance**: If one server fails, others continue working

In [ ]:
%%capture --no-stderr
%pip install feedparser PyMuPDF mcp arxiv

### Arxiv MCP Server

In [ ]:
%%writefile arxiv_server.py

from fastmcp import FastMCP
import arxiv
from typing import List, Dict, Any

mcp = FastMCP(name="ArxivMCP")

@mcp.tool()
def search_arxiv(query: str, max_results: int = 5) -> List[Dict[str, Any]]:
  """Search ArXiv for the top n results of available research based on your query

  Args:
    query (str): The search query to find relevant ArXiv papers
    max_results (int): The total number of results to return

  Returns:
    list: A list of the top n papers that match the query
  """
  search = arxiv.Search(
    query=query,
    max_results=max_results,
    sort_by=arxiv.SortCriterion.SubmittedDate
  )
  results = list(search.results())
  results = [{
      "title": result.title,
      "summary": result.summary,
      "authors": [author.name for author in result.authors],
      "published": str(result.published),
      "updated": str(result.updated),
  } for result in results]
  return results


if __name__ == "__main__":
  mcp.run(transport="stdio")


Overwriting arxiv_server.py


### Weather MCP Server

In [ ]:
%%writefile weather_server.py
from mcp.server.fastmcp import FastMCP
import httpx
import os
from dotenv import load_dotenv
from fastapi import HTTPException
from datetime import datetime
import uvicorn
import logging

# Load environment variables from .env file
# load_dotenv()
WEATHER_API_KEY = "87af948f625a41298ee211736251807"#os.getenv("WEATHER_API_KEY")

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
logger = logging.getLogger("WeatherMCP")

# Create an MCP server named "AdvancedWeather"
mcp = FastMCP(name="WeatherMCP")

# Helper: call WeatherAPI asynchronously
def validate_date(dt_str: str) -> None:
    """
    Ensure date string is in YYYY-MM-DD format.
    Raises HTTPException if invalid.
    """
    try:
        datetime.strptime(dt_str, "%Y-%m-%d")
    except ValueError:
        raise HTTPException(status_code=400, detail=f"Invalid date: {dt_str}. Use YYYY-MM-DD.")

async def fetch(endpoint: str, params: dict) -> dict:
    """
    Perform async GET to WeatherAPI and return JSON.
    Raises HTTPException on errors.
    Enhanced: logs requests, handles non-JSON errors gracefully.
    """
    if not WEATHER_API_KEY:
        logger.error("Weather API key not set.")
        raise HTTPException(status_code=500, detail="Weather API key not set.")

    params["key"] = WEATHER_API_KEY
    url = f"https://api.weatherapi.com/v1/{endpoint}"
    logger.info(f"Requesting {url} with params {params}")
    async with httpx.AsyncClient() as client:
        try:
            resp = await client.get(url, params=params)
            try:
                data = resp.json()
            except Exception:
                data = None
            if resp.status_code != 200:
                detail = (data or {}).get("error", {}).get("message", resp.text)
                logger.error(f"WeatherAPI error {resp.status_code}: {detail}")
                raise HTTPException(status_code=resp.status_code, detail=detail)
            logger.info(f"WeatherAPI success: {url}")
            return data
        except httpx.RequestError as e:
            logger.error(f"HTTPX request error: {e}")
            raise HTTPException(status_code=500, detail=f"Request error: {e}")
        except Exception as e:
            logger.error(f"Unexpected error: {e}")
            raise HTTPException(status_code=500, detail=f"Unexpected error: {e}")


# MCP Tools

@mcp.tool()
async def weather_current(q: str, aqi: str = "no") -> dict:
    """
    Get current weather for a location.
    Args:
        q (str): Location query (city name, lat/lon, postal code, etc).
        aqi (str): Include air quality data ('yes' or 'no').
    Returns:
        dict: WeatherAPI current weather JSON.
    """
    if not q:
        raise HTTPException(status_code=400, detail="Location (q) is required.")
    return await fetch("current.json", {"q": q, "aqi": aqi})

@mcp.tool()
async def weather_forecast(
    q: str,
    days: int = 1,
    aqi: str = "no",
    alerts: str = "no"
) -> dict:
    """
    Get weather forecast (1–14 days) for a location.
    Args:
        q (str): Location query (city name, lat/lon, postal code, etc).
        days (int): Number of days (1–14).
        aqi (str): Include air quality ('yes' or 'no').
        alerts (str): Include weather alerts ('yes' or 'no').
    Returns:
        dict: WeatherAPI forecast JSON.
    """
    if not q:
        raise HTTPException(status_code=400, detail="Location (q) is required.")
    if days < 1 or days > 14:
        raise HTTPException(status_code=400, detail="'days' must be between 1 and 14.")
    return await fetch("forecast.json", {"q": q, "days": days, "aqi": aqi, "alerts": alerts})

@mcp.tool()
async def weather_history(q: str, dt: str) -> dict:
    """
    Get historical weather for a location on a given date (YYYY-MM-DD).
    Args:
        q (str): Location query.
        dt (str): Date in YYYY-MM-DD format.
    Returns:
        dict: WeatherAPI history JSON.
    """
    if not q:
        raise HTTPException(status_code=400, detail="Location (q) is required.")
    validate_date(dt)
    return await fetch("history.json", {"q": q, "dt": dt})

@mcp.tool()
async def weather_alerts(q: str) -> dict:
    """
    Get weather alerts for a location.
    Args:
        q (str): Location query.
    Returns:
        dict: WeatherAPI alerts JSON.
    """
    if not q:
        raise HTTPException(status_code=400, detail="Location (q) is required.")
    # Alerts come from forecast with alerts=yes
    return await fetch("forecast.json", {"q": q, "days": 1, "alerts": "yes"})

@mcp.tool()
async def weather_airquality(q: str) -> dict:
    """
    Get air quality for a location.
    Args:
        q (str): Location query.
    Returns:
        dict: WeatherAPI air quality JSON.
    """
    if not q:
        raise HTTPException(status_code=400, detail="Location (q) is required.")
    return await fetch("current.json", {"q": q, "aqi": "yes"})

@mcp.tool()
async def weather_astronomy(q: str, dt: str) -> dict:
    """
    Get astronomy data (sunrise, sunset, moon) for a date (YYYY-MM-DD).
    Args:
        q (str): Location query.
        dt (str): Date in YYYY-MM-DD format.
    Returns:
        dict: WeatherAPI astronomy JSON.
    """
    if not q:
        raise HTTPException(status_code=400, detail="Location (q) is required.")
    validate_date(dt)
    return await fetch("astronomy.json", {"q": q, "dt": dt})

@mcp.tool()
async def weather_search(q: str) -> dict:
    """
    Search for locations matching query.
    Args:
        q (str): Location query.
    Returns:
        dict: WeatherAPI search JSON.
    """
    if not q:
        raise HTTPException(status_code=400, detail="Location (q) is required.")
    return await fetch("search.json", {"q": q})

@mcp.tool()
async def weather_timezone(q: str) -> dict:
    """
    Get timezone info for a location.
    Args:
        q (str): Location query.
    Returns:
        dict: WeatherAPI timezone JSON.
    """
    if not q:
        raise HTTPException(status_code=400, detail="Location (q) is required.")
    return await fetch("timezone.json", {"q": q})

@mcp.tool()
async def weather_sports(q: str) -> dict:
    """
    Get sports events (e.g., football, cricket) for a location.
    Args:
        q (str): Location query.
    Returns:
        dict: WeatherAPI sports JSON.
    """
    if not q:
        raise HTTPException(status_code=400, detail="Location (q) is required.")
    return await fetch("sports.json", {"q": q})

# Run the MCP server
if __name__ == "__main__":
    mcp.run(transport="stdio")

Overwriting weather_server.py


### Smoke Test Client

In [ ]:
%%writefile client.py
# A client that smoketests our server
from fastmcp import Client
import asyncio

async def main(server_path: str):
  client = Client(server_path)
  async with client:
    await client.ping()
    tools = await client.list_tools()
    print(tools)

if __name__ == "__main__":
  import sys
  server_path = sys.argv[1]
  asyncio.run(main(server_path))

Overwriting client.py


To test the servers, in the cell below add the server file name at the end of the call.

Example:
```bash
python client.py weather_server.py
```

Or

```bash
python client.py arxiv_server.py
```

In [ ]:
%%bash

python client.py weather_server.py

[Tool(name='weather_current', title=None, description="\n    Get current weather for a location.\n    Args:\n        q (str): Location query (city name, lat/lon, postal code, etc).\n        aqi (str): Include air quality data ('yes' or 'no').\n    Returns:\n        dict: WeatherAPI current weather JSON.\n    ", inputSchema={'properties': {'q': {'title': 'Q', 'type': 'string'}, 'aqi': {'default': 'no', 'title': 'Aqi', 'type': 'string'}}, 'required': ['q'], 'title': 'weather_currentArguments', 'type': 'object'}, outputSchema=None, annotations=None, meta=None), Tool(name='weather_forecast', title=None, description="\n    Get weather forecast (1–14 days) for a location.\n    Args:\n        q (str): Location query (city name, lat/lon, postal code, etc).\n        days (int): Number of days (1–14).\n        aqi (str): Include air quality ('yes' or 'no').\n        alerts (str): Include weather alerts ('yes' or 'no').\n    Returns:\n        dict: WeatherAPI forecast JSON.\n    ", inputSchema={'

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
2025-09-10 21:46:14,638 INFO Processing request of type PingRequest
2025-09-10 21:46:14,641 INFO Processing request of type ListToolsRequest


In [ ]:
%%bash

python client.py arxiv_server.py

[Tool(name='search_arxiv', title=None, description='Search ArXiv for the top n results of available research based on your query\n\nArgs:\n  query (str): The search query to find relevant ArXiv papers\n  max_results (int): The total number of results to return\n\nReturns:\n  list: A list of the top n papers that match the query', inputSchema={'properties': {'query': {'title': 'Query', 'type': 'string'}, 'max_results': {'default': 5, 'title': 'Max Results', 'type': 'integer'}}, 'required': ['query'], 'type': 'object'}, outputSchema={'properties': {'result': {'items': {'additionalProperties': True, 'type': 'object'}, 'title': 'Result', 'type': 'array'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta={'_fastmcp': {'tags': []}})]


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭────────────────────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___  _____           __  __  _____________    ____    ____     │
│       _ __ ___ .'____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \____/____/\__/_/  /_/\____/_/      /_____(*)____/      │
│                                                                            │
│                                            

In [ ]:
%%writefile multiserver_mcp_client.py

from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
import asyncio

client = MultiServerMCPClient(
    {
        "weather": {
            "command": "python",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": ["weather_server.py"],
            "transport": "stdio",
        },
        "arxiv": {
            "command": "python",
            "args": ["arxiv_server.py"],
            "transport": "stdio"
        }
    }
)

async def main(user_input: str):
  tools = await client.get_tools()
  agent = create_react_agent("openai:gpt-4.1-mini", tools)
  print("Welcome to the weather and arxiv AI agent. Ask a question about the weather or research.")
  message_template = {"messages": [user_input]}

  async for message in agent.astream(message_template):
    print(message)

if __name__ == "__main__":
  import sys
  user_input = sys.argv[1] if len(sys.argv) > 1 else "What is the weather in New York?"
  asyncio.run(main(user_input))


Overwriting multiserver_mcp_client.py


In [ ]:
%%bash

python multiserver_mcp_client.py "What is the latest research available on AI and what's the weather in New York like?"

Welcome to the weather and arxiv AI agent. Ask a question about the weather or research.
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Y24jaUI54PwEmt1ArfL7iVkf', 'function': {'arguments': '{"query": "Artificial Intelligence", "max_results": 5}', 'name': 'search_arxiv'}, 'type': 'function'}, {'id': 'call_w5YdUzjMcQFuQnAdbUozvj5S', 'function': {'arguments': '{"q": "New York"}', 'name': 'weather_current'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 688, 'total_tokens': 740, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4fce0778af', 'id': 'chatcmpl-CEMw7tzj3yR8ZzJ5roTSdYtfyd7eO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs':

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭────────────────────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___  _____           __  __  _____________    ____    ____     │
│       _ __ ___ .'____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \____/____/\__/_/  /_/\____/_/      /_____(*)____/      │
│                                                                            │
│                                            

## 4.9 Key Takeaways and Production Deployment

### What You've Accomplished

Congratulations! You've mastered the complete MCP ecosystem and built production-ready integrations:

✅ **MCP Protocol Understanding** - Learned the fundamentals of Model Context Protocol  
✅ **FastMCP Server Development** - Built comprehensive servers with proper validation  
✅ **LangGraph Agent Integration** - Created real agents using official MCP adapters  
✅ **Production Deployment** - Standalone servers supporting multiple transports  
✅ **Multi-Server Architectures** - Understanding of complex agent workflows  
✅ **Security and Best Practices** - Error handling, validation, and safe operations  

### Complete Workflow Mastered

```
1. 🔧 Design MCP Server    → Define tools, resources, and business logic
2. 📝 Implement with FastMCP → Use decorators and type hints for clean APIs  
3. 🧪 Test Thoroughly      → Validate all edge cases and error conditions
4. 📦 Package for Deployment → Create standalone executables
5. 🤖 Integrate with Agents → Use MultiServerMCPClient and create_react_agent
6. 🚀 Deploy to Production → Support HTTP transport, monitoring, scaling
```

### Technical Skills Developed

**MCP Server Development:**
- Tool registration with `@mcp.tool()` decorator
- Automatic JSON schema generation from type hints
- Comprehensive error handling and validation
- Resource management and server introspection
- Support for both stdio and HTTP transports

**LangGraph Integration:**
- **MultiServerMCPClient** configuration for multiple servers
- **Automatic tool discovery** from MCP servers  
- **ReAct agent creation** with `create_react_agent`
- **Natural language processing** of complex queries
- **Multi-step problem solving** using multiple tools

**Production Features:**
- Standalone server deployment patterns
- HTTP transport for production scalability
- Proper error handling at all integration layers
- Security considerations and input validation
- Monitoring and observability patterns

### Real-World Applications Unlocked

This MCP foundation enables you to build:

**Enterprise AI Systems:**
- Connect agents to databases, CRMs, and internal APIs
- Automate complex business workflows across multiple systems
- Provide AI access to company knowledge bases and documentation
- Build custom tool ecosystems for specific business domains

**Developer Tools and Automation:**
- AI coding assistants with access to version control and CI/CD
- Automated testing, deployment, and monitoring tools
- Integration with IDEs, build systems, and project management
- Code review and documentation generation systems

**Data Science and Analytics:**
- AI agents that query data warehouses and visualization tools
- Automated report generation and insight discovery
- Integration with ML pipelines and experiment tracking
- Real-time data analysis and decision support systems

### Multi-Server Agent Architectures

You now understand how to build sophisticated agents that combine tools from different domains:

```python
# Production multi-server configuration
server_config = {
    "math": {
        "command": "python",
        "args": ["/opt/mcp/math_server.py"],
        "transport": "stdio"
    },
    "weather": {
        "url": "http://weather-api:8001",
        "transport": "http",
        "timeout": 30
    },
    "database": {
        "url": "http://db-gateway:8002",
        "transport": "http",
        "auth": {"api_key": "${DB_API_KEY}"}
    },
    "analytics": {
        "command": "python",
        "args": ["/opt/mcp/analytics_server.py"],
        "transport": "stdio",
        "env": {"DATA_SOURCE": "${ANALYTICS_DB_URL}"}
    }
}

# Agent can seamlessly use tools from all servers
client = MultiServerMCPClient(server_config)
agent = create_react_agent(llm, await client.get_tools())
```

### Next Steps in Your MCP Journey

**Immediate Actions:**
1. **Complete Lab 5A** - Build the comprehensive math operations server
2. **Complete Lab 5B** - Create the weather API server with caching
3. **Experiment with Multi-Server Setups** - Combine different tool types
4. **Deploy to Production** - Use HTTP transport and monitoring

**Advanced Exploration:**
- **Custom Transport Protocols** - Build WebSocket or gRPC transports
- **Authentication and Authorization** - Implement secure access patterns
- **Monitoring and Observability** - Add metrics, logging, and health checks
- **Performance Optimization** - Caching, connection pooling, load balancing
- **Enterprise Integration** - Connect to existing business systems

### Production Deployment Checklist

Before deploying MCP servers to production:

- [ ] **Security**: Authentication, input validation, rate limiting
- [ ] **Error Handling**: Graceful failures and detailed error messages  
- [ ] **Monitoring**: Health checks, metrics, and logging
- [ ] **Performance**: Caching, connection pooling, timeout handling
- [ ] **Documentation**: API documentation and usage examples
- [ ] **Testing**: Unit tests, integration tests, load testing
- [ ] **Deployment**: Container images, configuration management
- [ ] **Scaling**: Load balancing, horizontal scaling patterns

### The MCP Ecosystem Advantage

By mastering MCP, you're part of a growing ecosystem that enables:
- **Interoperability** - Tools work across different AI platforms
- **Reusability** - One server serves many agent applications
- **Scalability** - Efficient client-server architecture
- **Maintainability** - Clear separation of concerns
- **Innovation** - Focus on business logic, not integration plumbing

---

**🎉 Congratulations!** You're now equipped to build production-ready MCP servers and integrate them with sophisticated LangGraph agents. The foundation you've built here will serve you well as you create the next generation of AI-powered applications.

**Ready for hands-on implementation?** Continue to the lab exercises to put these concepts into practice with real code and working examples!